In [1]:
import numpy as np
import pickle
from sklearn.model_selection import train_test_split

In [2]:
class ProgBar:

    def __init__(self, n_elements,int_str):
        
        import sys

        self.n_elements = n_elements
        self.progress = 0

        print(int_str)

        # initiallizing progress bar

        info = '{:.2f}% - {:d} of {:d}'.format(0,0,n_elements)

        formated_bar = ' '*int(50)

        sys.stdout.write("\r")

        sys.stdout.write('[%s] %s' % (formated_bar,info))

        sys.stdout.flush()

    def update(self,prog_info=None):
        
        import sys

        if prog_info == None:

            self.progress += 1

            percent = (self.progress)/self.n_elements * 100 / 2

            info = '{:.2f}% - {:d} of {:d}'.format(percent*2,self.progress,self.n_elements)

            formated_bar = '-'* int (percent) + ' '*int(50-percent)

            sys.stdout.write("\r")

            sys.stdout.write('[%s] %s' % (formated_bar,info))

            sys.stdout.flush()


        else:

            self.progress += 1

            percent = (self.progress)/self.n_elements * 100 / 2

            info = '{:.2f}% - {:d} of {:d} '.format(percent*2,self.progress,self.n_elements) + prog_info

            formated_bar = '-'* int (percent) + ' '*int(50-percent)

            sys.stdout.write("\r")

            sys.stdout.write('[%s] %s' % (formated_bar,info))

            sys.stdout.flush()


In [3]:
good_tools = pickle.load(open("good_tools.pkl", "rb"))

bad_tools = pickle.load(open("bad_tools.pkl", "rb"))


In [4]:
n_time_series = [10,20,30,40,80,160,320]

bar = ProgBar(len(n_time_series)*33,"Creating data divisinons...")

for count,n in enumerate(n_time_series):

    n_good = n*0.8

    n_bad = n*0.2

    good_size = n_good/len(good_tools)

    bad_size = n_bad/len(bad_tools)

    for i in range(33):

        _, good_division = train_test_split(good_tools,
                                            test_size=good_size)

        _, bad_division = train_test_split(bad_tools,
                                        test_size=bad_size)

        division = np.vstack((good_division,bad_division))

        flattened_input = []

        for time_serie in division:

            flattened_input.append(np.ravel(time_serie))

        flattened_input = np.array(flattened_input)

        target = np.ones(len(division))

        target[:len(good_division)] = 0

        pickle.dump(flattened_input,open("/media/thiago/ANGELICA/data_divisions/division__{}__n_series__{}__{}__.pkl".format(count,n,i), "wb"), protocol=4)
        pickle.dump(target,open("/media/thiago/ANGELICA/data_divisions/target__{}__n_series__{}__{}__.pkl".format(count,n,i), "wb"))

        bar.update()


Creating data divisinons...
[--------------------------------------------------] 100.00% - 231 of 231